In [5]:
pip install pyarabic

In [8]:
pip install nltk

In [9]:
import nltk
from nltk.corpus import wordnet
import re
from nltk.stem import ISRIStemmer
import pyarabic.araby as araby

In [11]:
 nltk.download('wordnet')
 nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [14]:
# Definitions for verb and noun inflections
inflectionVerbs = [('E', 'ت'), ('S', 'ت'), ('S', 'ي'), ('S', 'سي'), ('S', 'سأ'),
                    ('B', 'ي', 'ا'), ('S', 'ن'), ('B', 'ي', 'ون'), ('B', 'ي', 'ان'),
                    ('B', 'ت', 'ون'), ('B', 'ت', 'ان'), ('E', 'نا'), ('E', 'تم'), ('E', 'وا'),
                    ('S', 'أل'), ('B', 'سي', 'ون'), ('B', 'سي', 'ان'), ('B', 'سي', 'ن'), ('B', 'ت', 'ن'),
                    ('B', 'ي', 'ن'), ('S', 'أن')]
inflectionsNouns = [('E', 'ا'), ('E', 'ان'), ('E', 'ين'), ('E', 'ون'), ('S', 'ال')]
stemmer = ISRIStemmer()

# Functions for deriving words and inflecting verbs and nouns
def derive_words(lemma):
    derived_words = []
    if len(lemma) == 3:
        # Process for three-letter roots
        derived_words.extend([
            lemma[0] + "ا" + lemma[1:],
            'م' + lemma[0:2] + 'و' + lemma[-1],
            'م' + lemma[0] + 'ا' + lemma[1] + lemma[2],
            lemma[0:2] + 'ي' + lemma[-1],
            "م" + lemma,
            "الم" + lemma,
            'ت' + lemma[0] + 'ا' + lemma[1] + lemma[2],
            lemma + "ة",
            lemma + "ي"
        ])
    if lemma[0] == 'ي':
        # Additional processing for roots starting with 'ي'
        base = lemma[1:len(lemma) - 2]
        last_two = lemma[-2:]
        derived_words.extend([
            'مُ' + base + last_two[0] + "َ" + last_two[1],
            'مُ' + base + last_two[0] + 'ِ' + last_two[1]
        ])
    return derived_words

def inflect_verbs(lemma):
    inflected_verbs = []
    if lemma[-1] == 'ى':
        lemma = lemma[:-1] + 'ي'
    for inflection in inflectionVerbs:
        if inflection[0] == 'S':
            inflected_verbs.append(inflection[1] + lemma)
        elif inflection[0] == 'E':
            inflected_verbs.append(lemma + inflection[1])
        elif inflection[0] == 'B':
            inflected_verbs.append(inflection[1] + lemma + inflection[2])
    return inflected_verbs

def inflect_nouns(lemma):
    inflected_nouns = []
    for inflection in inflectionsNouns:
        if inflection[0] == 'S':
            inflected_nouns.append(inflection[1] + lemma)
        elif lemma[-1] != 'ة' and inflection[0] == 'E':
            if inflection[1] == 'ون':
                stemmed_noun = stemmer.stem(lemma)
                after_filter = araby.strip_diacritics(lemma)
                if stemmed_noun == after_filter:
                    inflected_nouns.append(lemma + inflection[1])
            elif inflection[1] == 'ا':
                inflected_nouns.append(lemma + "ً" + inflection[1])
            else:
                inflected_nouns.append(lemma + inflection[1])
    return inflected_nouns


word = input("Please enter an english word: ").lower().strip()
word_type = input("Please enter the type of the word (v for verb || n for noun): ").lower().strip()
synsets = wordnet.synsets(word, pos=word_type)
print(f'Number of synsets found: {len(synsets)}')

set_of_used_lemmas = set()
for synset in synsets:
    lemmas_ar = synset.lemma_names('arb')
    for lemma in lemmas_ar:
        lemma = araby.strip_diacritics(lemma)
        if lemma in set_of_used_lemmas or re.search('_', lemma):
            continue
        if word_type == 'v':
            inflected_verbs = inflect_verbs(lemma)
            derived_words = derive_words(lemma)
            print(f"{lemma} has these inflections: {inflected_verbs}")
            if derived_words:
                print(f"{lemma} has these derivations: {derived_words}")
        elif word_type == 'n':
            inflected_nouns = inflect_nouns(lemma)
            print(f"{lemma} has these inflections: {inflected_nouns}")
        set_of_used_lemmas.add(lemma)


Please enter an english word: play
Please enter the type of the word (v for verb || n for noun): v
Number of synsets found: 35
بارى has these inflections: ['باريت', 'تباري', 'يباري', 'سيباري', 'سأباري', 'يباريا', 'نباري', 'يباريون', 'يباريان', 'تباريون', 'تباريان', 'بارينا', 'باريتم', 'باريوا', 'ألباري', 'سيباريون', 'سيباريان', 'سيبارين', 'تبارين', 'يبارين', 'أنباري']
تسلى has these inflections: ['تسليت', 'تتسلي', 'يتسلي', 'سيتسلي', 'سأتسلي', 'يتسليا', 'نتسلي', 'يتسليون', 'يتسليان', 'تتسليون', 'تتسليان', 'تسلينا', 'تسليتم', 'تسليوا', 'ألتسلي', 'سيتسليون', 'سيتسليان', 'سيتسلين', 'تتسلين', 'يتسلين', 'أنتسلي']
تلاعب has these inflections: ['تلاعبت', 'تتلاعب', 'يتلاعب', 'سيتلاعب', 'سأتلاعب', 'يتلاعبا', 'نتلاعب', 'يتلاعبون', 'يتلاعبان', 'تتلاعبون', 'تتلاعبان', 'تلاعبنا', 'تلاعبتم', 'تلاعبوا', 'ألتلاعب', 'سيتلاعبون', 'سيتلاعبان', 'سيتلاعبن', 'تتلاعبن', 'يتلاعبن', 'أنتلاعب']
تنافس has these inflections: ['تنافست', 'تتنافس', 'يتنافس', 'سيتنافس', 'سأتنافس', 'يتنافسا', 'نتنافس', 'يتنافسون', 'يتن